In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
read = pd.read_csv("data/READCopyProtein50.csv")
coad = pd.read_csv("data/COADCopyProtein50.csv")
gse = pd.read_csv("data/GSE62254CopyConvertedProtein.csv")
all_data = pd.concat((read, coad, gse))

In [3]:
for df in [read, coad, gse, all_data]:
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.dropna(axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [4]:
from sklearn import model_selection
from sklearn import linear_model

In [5]:
X = all_data.drop(["COVAR_M", "COVAR_N_status"], axis=1)
y = all_data["COVAR_N_status"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=17)

In [6]:
model = linear_model.LogisticRegression(solver='saga', max_iter=1000)

In [7]:
distributions = dict(C=range(1, 5),penalty=['l2', 'l1'])

In [8]:
clf = model_selection.GridSearchCV(model, distributions, verbose=2)

In [9]:
search = clf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/ZanderL/opt/anaconda3/envs/cs229project/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ZanderL/opt/anaconda3/envs/cs229project/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ZanderL/opt/anaconda3/envs/cs229project/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ZanderL/opt/anaconda3/envs/cs229project/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ZanderL/opt/anaconda3/envs/cs229project/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which 

/Users/ZanderL/opt/anaconda3/envs/cs229project/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [10]:
search.best_params_

{'C': 1, 'penalty': 'l2'}